In [1]:
from dotenv import load_dotenv
import openai, json

load_dotenv()

client = openai.OpenAI()

messages = [];

In [2]:
def get_weather(city):
  return f"The weather in 33 degrees celcius."

FUNCTION_MAP = {
  'get_weather': get_weather
}


In [3]:
TOOLS = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "A function to get the weather of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "The name of the city to get the weather of"
          }
        },
        "required": ["city"]
      }
    }
  }
]

In [4]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
  if message.tool_calls:
    messages.append(
      {
        "role": "assistant",
        "content": message.content or "",
        "tool_calls": [
          {
            "id": tool_call.id,
            "type": "function",
            "function": {
              "name": tool_call.function.name,
              "arguments": tool_call.function.arguments
            }
          } 
          for tool_call in message.tool_calls
        ],
      }
    )

    for tool_call in message.tool_calls:
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments

      print(f"Calling finction: {function_name} with arguments: {arguments}")

      try:
        # python dict로 변환
        arguments = json.loads(arguments)
      except json.JSONDecodeError:
        arguments = {}

      function_to_run = FUNCTION_MAP.get(function_name)

      # arguments는 dict니, dict로 변환
      # ** 연산자: dictionary를 함수의 keyword arguments로 풀어서 전달
      # 예: arguments = {"city": "Seoul"} 일 때
      #     function_to_run(**arguments)는 function_to_run(city="Seoul")과 동일
      result = function_to_run(**arguments)

      print(f"Ran {function_name} with arguments {arguments} for a result of {result}")

      messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": result,
      })

    call_ai()
  else:
    messages.append({"role": "assistant", "content": message.content})
    print(f"AI: {message.content}")

def call_ai():
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=TOOLS,
  )
  process_ai_response(response.choices[0].message)

In [5]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: my name is jsj
AI: Nice to meet you, jsj! How can I assist you today?
User: what is my name
AI: Your name is jsj.
User: what is the weather in seoul
Calling finction: get_weather with arguments: {"city":"Seoul"}
Ran get_weather with arguments {'city': 'Seoul'} for a result of The weather in 33 degrees celcius.
AI: The weather in Seoul is currently 33 degrees Celsius.


In [6]:
messages

[{'role': 'user', 'content': 'my name is jsj'},
 {'role': 'assistant',
  'content': 'Nice to meet you, jsj! How can I assist you today?'},
 {'role': 'user', 'content': 'what is my name'},
 {'role': 'assistant', 'content': 'Your name is jsj.'},
 {'role': 'user', 'content': 'what is the weather in seoul'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_iFOJCpMOOaKeNL9q4ttUleZm',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_iFOJCpMOOaKeNL9q4ttUleZm',
  'name': 'get_weather',
  'content': 'The weather in 33 degrees celcius.'},
 {'role': 'assistant',
  'content': 'The weather in Seoul is currently 33 degrees Celsius.'}]